In [1]:
import os
import glob
import time
import torch
import yaml
from ultralytics import YOLO

# --- CONFIGURATION ---
BASE_PATH = r"C:\Users\ADMIN\Documents\Dev_Projects\Traffic_Infosys\UVH-26_dataset"
YAML_FILE = "uvh26_final.yaml"
MODEL_TYPE = "yolov9e.pt"
PROJECT_NAME = "UVH_Traffic_Project"
RUN_NAME = "yolov9e_a6000_run"

# Training Settings (A6000)
BATCH_SIZE = 16  
IMG_SIZE = 640   
EPOCHS = 40      
WORKERS = 8      # Reduced to 8 to be safe on Windows

In [2]:
def fix_dataset_structure_and_yaml():
    """
    1. Renames 'data' -> 'images' and 'yolo_labels' -> 'labels'
    2. Updates the YAML file to point to 'images'
    3. Clears old cache files
    """
    print("\n--- Checking Dataset Structure ---")
    subsets = ["UVH-26-Train", "UVH-26-Val"]
    structure_fixed = False

    for subset in subsets:
        subset_path = os.path.join(BASE_PATH, subset)
        
        # Define Paths
        old_img = os.path.join(subset_path, "data")
        new_img = os.path.join(subset_path, "images")
        old_lbl = os.path.join(subset_path, "yolo_labels")
        new_lbl = os.path.join(subset_path, "labels")

        # Rename Image Folder
        if os.path.exists(old_img) and not os.path.exists(new_img):
            try:
                os.rename(old_img, new_img)
                print(f"[{subset}] Renamed 'data' -> 'images'")
                structure_fixed = True
            except PermissionError:
                print(f"ERROR: Close any folders/files open in {old_img} and try again.")
                return False
        
        # Rename Label Folder
        if os.path.exists(old_lbl) and not os.path.exists(new_lbl):
            try:
                os.rename(old_lbl, new_lbl)
                print(f"[{subset}] Renamed 'yolo_labels' -> 'labels'")
            except PermissionError:
                print(f"ERROR: Close any folders/files open in {old_lbl} and try again.")
                return False
        
        # Delete Cache (Forces YOLO to re-scan labels)
        if os.path.exists(new_img):
            cache_files = glob.glob(os.path.join(new_img, "*.cache"))
            for cache in cache_files:
                try:
                    os.remove(cache)
                    print(f"[{subset}] Deleted old cache file.")
                except:
                    pass

    # Update YAML File to point to "images" instead of "data"
    if os.path.exists(YAML_FILE):
        with open(YAML_FILE, 'r') as f:
            content = f.read()
        
        if "data" in content:
            print("Updating YAML file paths...")
            new_content = content.replace("/data", "/images").replace("\\data", "\\images")
            with open(YAML_FILE, 'w') as f:
                f.write(new_content)
            print("YAML updated successfully.")
            
    return True

In [3]:
def cooling_callback(trainer):
    """Pauses for 1 minute after every epoch to cool GPU."""
    current_epoch = trainer.epoch + 1
    if current_epoch > 0:
        print(f"\n[COOLING] Epoch {current_epoch} complete. Pausing for 60s...")
        time.sleep(60)
        print("[COOLING] Resuming...\n")

In [4]:
import os
import glob
from tqdm import tqdm

# --- CONFIGURATION ---
BASE_PATH = r"C:\Users\ADMIN\Documents\Dev_Projects\Traffic_Infosys\UVH-26_dataset"
TRAIN_LBL_DIR = os.path.join(BASE_PATH, "UVH-26-Train", "labels")
VAL_LBL_DIR = os.path.join(BASE_PATH, "UVH-26-Val", "labels")

# We want to keep classes 0 to 12.
# Class 13 is "Other" (Agricultural/Specialized), which we are REMOVING.
VALID_IDS = set(range(13))  # {0, 1, 2, ..., 12}

def clean_other_class(directory):
    print(f"Scanning for 'Other' class (ID 13) in: {directory}...")
    txt_files = glob.glob(os.path.join(directory, "**", "*.txt"), recursive=True)
    
    files_modified = 0
    instances_removed = 0
    
    for file_path in tqdm(txt_files):
        if os.path.getsize(file_path) == 0: continue
        
        with open(file_path, 'r') as f:
            lines = f.readlines()
            
        new_lines = []
        file_changed = False
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                try:
                    class_id = int(parts[0])
                    
                    if class_id in VALID_IDS:
                        new_lines.append(line)
                    else:
                        # This matches ID 13 (Other) or anything higher
                        file_changed = True
                        instances_removed += 1
                except ValueError:
                    pass
        
        # Save back only if we removed something
        if file_changed:
            with open(file_path, 'w') as f:
                f.writelines(new_lines)
            files_modified += 1
            
    print(f"-> Removed {instances_removed} 'Other' instances from {files_modified} files.")

# 1. Run Cleaning
clean_other_class(TRAIN_LBL_DIR)
clean_other_class(VAL_LBL_DIR)

# 2. Clear Cache (CRITICAL)
print("\nClearing YOLO cache to enforce changes...")
for subset in ["UVH-26-Train", "UVH-26-Val"]:
    # Cache file sits in the 'images' folder
    cache_path = os.path.join(BASE_PATH, subset, "images") 
    for f in glob.glob(os.path.join(cache_path, "*.cache")):
        try:
            os.remove(f)
            print(f"Deleted cache: {f}")
        except:
            pass

print("\nSUCCESS: Class 13 ('Other') has been removed. You can now train!")

Scanning for 'Other' class (ID 13) in: C:\Users\ADMIN\Documents\Dev_Projects\Traffic_Infosys\UVH-26_dataset\UVH-26-Train\labels...


100%|███████████████████████████████████████████████████████████████████████████| 41978/41978 [00:55<00:00, 754.81it/s]


-> Removed 575 'Other' instances from 532 files.
Scanning for 'Other' class (ID 13) in: C:\Users\ADMIN\Documents\Dev_Projects\Traffic_Infosys\UVH-26_dataset\UVH-26-Val\labels...


100%|████████████████████████████████████████████████████████████████████████████| 5297/5297 [00:01<00:00, 4207.46it/s]

-> Removed 70 'Other' instances from 64 files.

Clearing YOLO cache to enforce changes...

SUCCESS: Class 13 ('Other') has been removed. You can now train!


In [5]:
def main():
    # 1. Fix Folders First
    if not fix_dataset_structure_and_yaml():
        print("Dataset structure check failed. Aborting.")
        return

    # 2. Check GPU
    if torch.cuda.is_available():
        vram = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"\nGPU Detected: {torch.cuda.get_device_name(0)} ({vram:.1f} GB)")
    else:
        print("WARNING: No GPU detected.")

    # 3. Load Model
    model = YOLO(MODEL_TYPE)

    # 4. Add Callback
    model.add_callback('on_train_epoch_end', cooling_callback)

    # 5. Train
    print("\nStarting Training...")
    model.train(
        data=YAML_FILE,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=0,
        workers=WORKERS,
        project=PROJECT_NAME,
        name=RUN_NAME,
        patience=10,
        save=True,
        exist_ok=True,
        verbose=True
    )
    print("Training Finished!")

if __name__ == '__main__':
    main()


--- Checking Dataset Structure ---
Updating YAML file paths...
YAML updated successfully.

GPU Detected: NVIDIA RTX 6000 Ada Generation (51.5 GB)

Starting Training...
New https://pypi.org/project/ultralytics/8.3.246 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.241  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 49140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=uvh26_final.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0